In [117]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns
import statsmodels.api as sm


In [118]:
def load_peace_sys_data():
    df = pd.DataFrame(pd.read_csv('peace_sys.csv', index_col=0, na_values=['(NA)']))
    return df

In [119]:
def get_odds_ratios(coefs):
    return np.exp(coefs[0])


In [120]:
def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    # first index refers to proba that belongs to class 0
    # second index refers to proba that belongs to calss 1
    p = model.predict_proba(x) # return matrix (N,2)
    # number of samples
    n = len(p)
    # number of features + 1 
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]]) # put intercept and coefs in same array
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1)) # 
    ans = np.zeros((m, m))
    for i in range(n):
        # dot product of transposed row and row
        # then multiply by both probas
        # add it to ans
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    # acovariance matrix
    vcov = np.linalg.inv(np.matrix(ans))
    # square root diagonal of covariace matrix
    se = np.sqrt(np.diag(vcov))
    # divide coefs by standard error
    t =  coefs/se 
    # two tailed using normal dist
    p = (1 - stats.norm.cdf(abs(t))) * 2
    return p

In [121]:
def logit_with_nan(data):
    data_copy = data.drop('PSys', axis=1)
    pvalues = []
    logit_beta = []
    logit_odds_ratio = []

    
    for label in data_copy.columns:
        X = data[[label, 'PSys']]
        X = X.dropna()
        
        y = X.PSys
        X = X.drop('PSys', axis=1)
        
        logreg = LogisticRegression(random_state=42, C=1e9)
        logreg.fit(X, y)
        pvalue = logit_pvalue(logreg, X)[1]
        
        pvalues.append(pvalue)
        
        beta = np.absolute(logreg.coef_[0][0])
        logit_beta.append(beta)
        logit_odds_ratio.append(np.exp(beta))
        
        
    res = pd.DataFrame({'Variable' : data_copy.columns,
                        'Logistic_Sig_Level' : pvalues,
                        'Logistic_Beta' : logit_beta,
                        'Logistic_Odds_Ratio' : logit_odds_ratio
                       })
    
    res = res.set_index('Variable')
    
    return res

In [122]:
def compute_ttest(peaceful, non_peaceful):
    variables = peaceful.columns
    res_df = pd.DataFrame(columns=['feature', 't_statistic', 'p_value'])
    
    for var in variables:
        x = peaceful[var]
        y = non_peaceful[var]
        ttest = stats.ttest_ind(x, y)
        temp = pd.DataFrame({'feature' : [var] , 
                             't_statistic' : ttest.statistic, 
                             'p_value' : [ttest.pvalue]})
        res_df = res_df.append(temp)
        
    res_df.set_index('feature', inplace=True)
    return res_df

In [123]:
def ttest_with_nan(data):
    """ 
    This is a two sided test for the null hypothesis that 2 independent samples
    have identical average(expected) values.
    
    Test assusmes that the populations have identical variances by default
    
    Parameters
    ----------
    data: DataFrame
        The DataFrame that contains Nan values
        
    Returns
    -------
    returns DataFrame that contains statistic and pvalue for every column in data
    
    t_statistic: float
        Calculated t-statistic
    
    p_value: float
        Two tailed p-value
            
    """
    # create a copy that excludes target variable
    data_copy = data.drop('PSys', axis=1)
    # prepare result data frame
    res_df = pd.DataFrame(columns=['Variable', 't_statistic', 'ttest_p_value'])
    # for every label, we separate each row by what its target value is
    # we drop the nan values
    for label in data_copy.columns:
        X = data[[label, 'PSys']]
        X = X.dropna()
        # filter peaceful 
        peaceful = X[X.PSys == 1]
        peaceful = peaceful.drop('PSys', axis=1)
        
        non_peaceful = X[X.PSys == 2]
        non_peaceful = non_peaceful.drop('PSys', axis=1) 
        # calculate two-sided ttest
        ttest_result = stats.ttest_ind(peaceful, non_peaceful)
        ttest_df = pd.DataFrame({'Variable' : [label],
                                 't_statistic' : ttest_result.statistic,
                                 'ttest_p_value' : ttest_result.pvalue
                                })
        
        res_df = res_df.append(ttest_df)
    
    res_df.set_index('Variable', inplace=True)
    
    return res_df

In [124]:
def compute_mannwhitneyu(peaceful, non_peaceful):
    variables = peaceful.columns
    res_df = pd.DataFrame(columns=['feature', 'statistic', 'p_value'])
    
    for var in variables:
        x = peaceful[var]
        y = non_peaceful[var]
        utest = stats.mannwhitneyu(x, y)
        temp = pd.DataFrame({'feature' : [var] , 
                             'statistic' : utest.statistic, 
                             'p_value' : [utest.pvalue]})
        res_df = res_df.append(temp)
        
    res_df.set_index('feature', inplace=True)
    return res_df

In [125]:
def mannwhitneyu_with_nan(data):
    """ 
    Computes the Mann-Whitney rank test on each of the columns of data
    Each column is seperated into 2 different arrays
    
    One group is peaceful, the other group is non-peaceful
    
    Parameters
    ----------
    data : DataFrame
        DataFrame that contains nan values
        
    Returns:
    -------
    Retures a DataFrame with U_statistic and Pvalue for every column
    
    U_statististic: float
        Mann-Whitney U statistic, equal to min(U for x, and U for y)
        
    p_value: float
        p-value assuming asymptotic normal distribution. Two-Sided.
    """
    data_copy = data.drop('PSys', axis=1)
    res_df = pd.DataFrame(columns=['Variable', 'U_statistic', 'mannwhitneyu_p_value'])
    for label in data_copy.columns:
        # isolate the target variable and current variable
        X = data[[label, 'PSys']]
        # drop all Nan values
        X = X.dropna()
        # filter out peaceful rows
        peaceful = X[X.PSys == 1]
        peaceful = peaceful.drop('PSys', axis=1)
        # filter out non peaceful rows
        non_peaceful = X[X.PSys == 2]
        non_peaceful = non_peaceful.drop('PSys', axis=1) 
        # calculates two-sided Mann-Whitney U test
        utest_result = stats.mannwhitneyu(peaceful, non_peaceful, alternative='two-sided')
        ttest_df = pd.DataFrame({'Variable' : [label],
                                 'U_statistic' : utest_result.statistic,
                                 'mannwhitneyu_p_value' : utest_result.pvalue
                                })
        
        res_df = res_df.append(ttest_df)
    
    res_df.set_index('Variable', inplace=True)
    
    return res_df

In [126]:
peace_sys = load_peace_sys_data()

peace_sys

,SCCS,Coder,ID1.1Over,ID1.2Ethno,Int2.1Mar,Int2.2Econ,Int2.3Pol,Int2.4Hist,Dep3.1Sec,Dep3.2Ecol,...,CM8.5Peace,CM8.6War,Lead9.1P,Lead9.2War,Cult10.1Com,Cult10.2Diff,Comp10.3,InComp10.4,PSys,PSysRec
Name,,,,,,,,,,,,,,,,,,,,,
Gilbertese,107,KA,9.0,2.0,3,9.0,9,9,9,9,...,9,9,9,9,9,9,9,9,2,0
Marshallese,108,EC,1.0,9.0,9,1.0,9,9,9,9,...,9,2,9,4,9,9,9,9,2,0
E. Pomo,135,"KA, EC",9.0,9.0,9,9.0,9,9,1,1,...,9,9,2,9,9,9,9,9,2,0
Popoluca,154,KA,9.0,3.0,9,3.0,9,9,9,3,...,9,9,9,9,9,9,9,9,2,0
Konso,35,KA,9.0,2.0,3,4.0,9,4,9,2,...,9,9,9,9,3,9,3,3,2,0
Bribri,157,DG,9.0,2.0,2,9.0,9,9,9,9,...,9,9,9,9,9,9,9,9,2,0
Tallensi,23,DG,1.0,2.0,4,9.0,1,3,3,9,...,2,9,9,9,4,1,3,9,2,0
Russians,54,DG,4.0,9.0,1,9.0,9,9,9,9,...,2,1,1,9,2,9,9,9,2,0
Trukese,109,DG,3.0,9.0,4,4.0,3,9,3,9,...,9,9,9,9,9,9,9,9,2,0


In [127]:
peace_sys = peace_sys.replace(9, np.nan)

In [128]:
peace_sys = peace_sys.drop(['SCCS','Coder'], axis=1)

In [129]:
NON_WAR_VARS = ['SymP6', 'NWNorm5.1', 'RitP6', 'Dep3.3Econ', 'Int2.4Hist', 'ID1.1Over', 
                'NWVal4.1', 'Int2.2Econ', 'Dep3.2Ecol', 'CM8.5Peace', 'PSys']

In [130]:
WAR_VARS = ['WNorm5.2', 'Lead9.2War', 'SymWar6', 'RitWar6', 'WVal4.2', 'PSys']

In [131]:
VARS = np.unique(NON_WAR_VARS + WAR_VARS)

In [132]:
VARS = np.roll(VARS, 6) # move PSys to the end of the array

In [133]:
VARS

array(['RitP6', 'RitWar6', 'SymP6', 'SymWar6', 'WNorm5.2', 'WVal4.2',
       'CM8.5Peace', 'Dep3.2Ecol', 'Dep3.3Econ', 'ID1.1Over',
       'Int2.2Econ', 'Int2.4Hist', 'Lead9.2War', 'NWNorm5.1', 'NWVal4.1',
       'PSys'], dtype='<U10')

In [134]:
X = peace_sys[VARS].drop('PSys', axis=1)
X = X.fillna(X.mean())

In [135]:
y = peace_sys.PSys
y = y.replace(2, 0)

# Logistic Regression

In [136]:
logit_results = logit_with_nan(peace_sys[VARS])

In [137]:
logit_results

,Logistic_Sig_Level,Logistic_Beta,Logistic_Odds_Ratio
Variable,,,
RitP6,0.013339,2.071619,7.937661
RitWar6,0.010238,1.619016,5.048121
SymP6,0.036682,2.573926,13.117217
SymWar6,0.040487,1.575338,4.832373
WNorm5.2,0.005159,1.323790,3.757636
WVal4.2,0.003350,1.668282,5.303049
CM8.5Peace,0.932156,8.542179,5126.503390
Dep3.2Ecol,0.031614,0.933155,2.542519
Dep3.3Econ,0.010404,1.845498,6.331255


# T-Test

In [138]:
ttest = ttest_with_nan(peace_sys[VARS])

In [139]:
ttest

,t_statistic,ttest_p_value
Variable,,
RitP6,3.893138,0.000839
RitWar6,-3.871833,0.000949
SymP6,3.926992,0.002364
SymWar6,-2.600997,0.018061
WNorm5.2,-3.758143,0.000768
WVal4.2,-4.723962,0.000039
CM8.5Peace,4.147288,0.003221
Dep3.2Ecol,2.502553,0.020254
Dep3.3Econ,4.070873,0.000367


# Mann-Whitney U-Test

In [140]:
mannwhitneyu = mannwhitneyu_with_nan(peace_sys[VARS])

In [141]:
mannwhitneyu

,U_statistic,mannwhitneyu_p_value
Variable,,
RitP6,115.5,0.001604
RitWar6,15.5,0.003076
SymP6,38.0,0.010393
SymWar6,21.0,0.033359
WNorm5.2,43.0,0.001794
WVal4.2,58.5,0.000786
CM8.5Peace,20.0,0.018906
Dep3.2Ecol,109.0,0.026365
Dep3.3Econ,174.0,0.001851


#  Random Forest

In [142]:
rfc = RandomForestClassifier(random_state=42, n_estimators=2000).fit(X, y)

In [143]:
rfc_feature_importances = pd.DataFrame({'Variables' : X.columns, 'rfc_feature_importance' : rfc.feature_importances_})
rfc_feature_importances = rfc_feature_importances.set_index('Variables')

In [144]:
rfc_feature_importances['rfc_feature_importance']

Variables
RitP6         0.104735
RitWar6       0.031976
SymP6         0.039809
SymWar6       0.017110
WNorm5.2      0.095222
WVal4.2       0.089838
CM8.5Peace    0.040647
Dep3.2Ecol    0.045090
Dep3.3Econ    0.084430
ID1.1Over     0.060565
Int2.2Econ    0.057519
Int2.4Hist    0.070696
Lead9.2War    0.021585
NWNorm5.1     0.161866
NWVal4.1      0.078914
Name: rfc_feature_importance, dtype: float64

In [154]:
all_methods = pd.DataFrame({'Random_Forest': rfc_feature_importances['rfc_feature_importance'],
                          'MannWhitneyU_Test' : mannwhitneyu['mannwhitneyu_p_value'],
                          'T_Test' : ttest['ttest_p_value'],
                          'Logistic_Regression' : logit_results['Logistic_Sig_Level'],
                           },)

In [155]:
all_methods

,Random_Forest,MannWhitneyU_Test,T_Test,Logistic_Regression
RitP6,0.104735,0.001604,0.000839,0.013339
RitWar6,0.031976,0.003076,0.000949,0.010238
SymP6,0.039809,0.010393,0.002364,0.036682
SymWar6,0.017110,0.033359,0.018061,0.040487
WNorm5.2,0.095222,0.001794,0.000768,0.005159
WVal4.2,0.089838,0.000786,0.000039,0.003350
CM8.5Peace,0.040647,0.018906,0.003221,0.932156
Dep3.2Ecol,0.045090,0.026365,0.020254,0.031614
Dep3.3Econ,0.084430,0.001851,0.000367,0.010404
ID1.1Over,0.060565,0.003749,0.001835,0.007980


In [157]:
all_methods['SUM'] = all_methods.sum(axis=1)

In [159]:
all_methods['PRODUCT'] = all_methods.product(axis=1)

In [160]:
all_methods

,Random_Forest,MannWhitneyU_Test,T_Test,Logistic_Regression,SUM,PRODUCT
RitP6,0.104735,0.001604,0.000839,0.013339,0.120516,2.264139e-10
RitWar6,0.031976,0.003076,0.000949,0.010238,0.046238,4.418896e-11
SymP6,0.039809,0.010393,0.002364,0.036682,0.089247,3.202169e-09
SymWar6,0.017110,0.033359,0.018061,0.040487,0.109017,4.550061e-08
WNorm5.2,0.095222,0.001794,0.000768,0.005159,0.102943,6.969107e-11
WVal4.2,0.089838,0.000786,0.000039,0.003350,0.094013,8.686053e-13
CM8.5Peace,0.040647,0.018906,0.003221,0.932156,0.994929,2.295542e-06
Dep3.2Ecol,0.045090,0.026365,0.020254,0.031614,0.123324,9.387540e-08
Dep3.3Econ,0.084430,0.001851,0.000367,0.010404,0.097052,5.787830e-11
ID1.1Over,0.060565,0.003749,0.001835,0.007980,0.074128,2.464029e-10
